# Outage probability versus $\gamma_{th}$

## Loading libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from statsmodels.stats.proportion import proportion_confint

# from misc import blocked_rays_main_paths


In [2]:
def blocked_rays_main_paths(x, y, x1, x2, y1, y2, r, DS):
    minx = np.min([x1, x2])
    maxx = np.max([x1, x2])
    miny = np.min([y1, y2])
    maxy = np.max([y1, y2])
    D3 = (((x >= minx) & (x <= maxx)) & ((y >= miny) & (y <= maxy))) & (DS <= r)               
    return D3

## Simulation settings

In [3]:
# Room dimensions
Lx = 30
Ly = 20
Lz = 3
room_area = Lx * Ly

ris_center_x = 12
ris_center_y = 0
ris_center_z = 1.5

N = 8

# Transmitter and receiver cooordinates
Tx = np.zeros(3)
Tx[0] = 8
Tx[1] = 3
Tx[2] = 1

Rx = np.zeros(3)
Rx[0] = 15
Rx[1] = 10
Rx[2] = 2.50

trials = 100000
GHz  = 18 # operating frequency
c    = 299792458  # speed of light
r    = 0.1  # obtruction radius
lmbd = 2.50  # intensity of the PPP (m^(-2))
sgm2 = 1.658e-11
Pt   = 0.1

# Table 2 (paper)
if (GHz == 18):
    z  = 130
    AO = 0.00006
    f  = 18e9
if (GHz == 26):
    z  = 200
    AO = 0.00013
    f  = 26e9
if (GHz == 60):
    z  = 390
    AO = 0.015
    f  = 60e9
if (GHz == 73):
    z  = 420
    AO = 0.0075
    f  = 73e9

wvlen= c / f

# Antenna gain
GtdB = 20 # Gain at the transmitter in dB   
GrdB = 10 # Gain at the receiver in dB
Gt   = np.power(10, GtdB/10)
Gr   = np.power(10, GrdB/10)
G    = Gr * Gt
B    = sgm2/(Pt * G)

## Room, RIS, Tx, and Rx

In [4]:
dx = wvlen/2
dz = wvlen/2

# RIS on the bottom wall
if (ris_center_y == 0):
    x_RIS  = dx * (np.linspace(1-N/2,N/2,N) - 0.50) + ris_center_x
    y_RIS  = np.zeros(N)
    z_RIS  = dz * (np.linspace(1-N/2,N/2,N) - 0.50) + ris_center_z
    wall = 'bottom'
# RIS on the left wall
if (ris_center_x == 0):
    y_RIS  = dx * (np.linspace(1-N/2,N/2,N) - 0.50) + ris_center_y
    x_RIS  = np.zeros(N)
    z_RIS  = dz * (np.linspace(1-N/2,N/2,N) - 0.50) + ris_center_z
    wall = 'left'
# RIS on the upper wall
if (ris_center_y == Ly):
    x_RIS  = dx * (np.linspace(1-N/2,N/2,N) - 0.50) + ris_center_x    # RIS x-coordinates
    y_RIS  = np.ones(N) * Ly
    z_RIS  = dz * (np.linspace(1-N/2,N/2,N) - 0.50) + ris_center_z
    wall = 'upper'
# RIS on the right wall
if (ris_center_x == Lx):
    y_RIS  = dx * (np.linspace(1-N/2,N/2,N) - 0.50) + ris_center_y     # RIS x-coordinates
    x_RIS  = np.ones(N) * Lx
    z_RIS  = dz * (np.linspace(1-N/2,N/2,N) - 0.50) + ris_center_z
    wall = 'right'
    
x_RIS, z_RIS = np.meshgrid(x_RIS, z_RIS)
y_RIS = np.zeros(z_RIS.shape)

c_RIS = np.zeros((N*N,3))
c_RIS[:,0] = x_RIS.reshape(-1)
c_RIS[:,1] = np.zeros(N*N)
c_RIS[:,2] = z_RIS.reshape(-1)

v1 = c_RIS - Tx
v2 = Rx - c_RIS
v3 = c_RIS - Rx

v1d = v1 /np.linalg.norm(v1,axis=1).reshape(-1,1)
v1e = np.zeros((1,3,N*N))
v1e[0,0,:] = v1d[:,0]
v1e[0,1,:] = v1d[:,1]
v1e[0,2,:] = v1d[:,2]

v2d = v2 /np.linalg.norm(v2,axis=1).reshape(-1,1)
v3d = v3 /np.linalg.norm(v3,axis=1).reshape(-1,1)

## Free path loss

In [5]:
# Distances
dk0     = np.linalg.norm(Rx - Tx)
dRIS    = np.linalg.norm(v1,axis=0) + np.linalg.norm(v2,axis=0)

# Free space path loss for LOS path and reflection paths
Lk      = 20 * np.log10(4 * np.pi * f * dk0 / c) + AO * dk0  # Eq. (1)
print('--------------------------------')
print('\t\t LOS')
print('distances[m]\t %.5f\t'%(dk0))
print('Lk[dB]\t\t %.5f\t'%(Lk))
print('--------------------------------')

--------------------------------
		 LOS
distances[m]	 10.01249	
Lk[dB]		 77.56468	
--------------------------------


## Number of obstructions per path and obstruction length per path

In [ ]:
start_time = time.time()

num_obs_per_path     = np.zeros((1,trials))
num_obs_per_path_RIS = np.zeros((N*N,trials))
obs_len_per_path     = np.zeros((1,trials))
obs_len_per_path_RIS = np.zeros((N*N,trials))

xmin = np.min([Tx[0], Rx[0]]) - r
xmax = np.max([Tx[0], Rx[0]]) + r
ymin = np.min([Tx[1], Rx[1]]) - r
ymax = np.max([Tx[1], Rx[1]]) + r
zmin = np.min([Tx[2], Rx[2]]) - r
zmax = np.max([Tx[2], Rx[2]]) + r

xTmin = np.minimum(np.repeat(Tx[0],N*N), c_RIS[:,0]).reshape(N*N,1) - r
xTmax = np.maximum(np.repeat(Tx[0],N*N), c_RIS[:,0]).reshape(N*N,1) + r
yTmin = np.minimum(np.repeat(Tx[1],N*N), c_RIS[:,1]).reshape(N*N,1) - r
yTmax = np.maximum(np.repeat(Tx[1],N*N), c_RIS[:,1]).reshape(N*N,1) + r
zTmin = np.minimum(np.repeat(Tx[2],N*N), c_RIS[:,2]).reshape(N*N,1) - r
zTmax = np.maximum(np.repeat(Tx[2],N*N), c_RIS[:,2]).reshape(N*N,1) + r

xRmin = np.minimum(np.repeat(Rx[0],N*N), c_RIS[:,0]).reshape(N*N,1) - r
xRmax = np.maximum(np.repeat(Rx[0],N*N), c_RIS[:,0]).reshape(N*N,1) + r
yRmin = np.minimum(np.repeat(Rx[1],N*N), c_RIS[:,1]).reshape(N*N,1) - r
yRmax = np.maximum(np.repeat(Rx[1],N*N), c_RIS[:,1]).reshape(N*N,1) + r
zRmin = np.minimum(np.repeat(Rx[2],N*N), c_RIS[:,2]).reshape(N*N,1) - r
zRmax = np.maximum(np.repeat(Rx[2],N*N), c_RIS[:,2]).reshape(N*N,1) + r

inv_normv1 = np.divide(1, np.linalg.norm(v1, axis=1))
inv_normv2 = np.divide(1, np.linalg.norm(v2, axis=1))

No = np.random.poisson(lam=lmbd*(xmax - xmin)*(ymax)*(zmax - zmin), size=trials) 
for ii in range(trials):
    p1 = np.zeros((No[ii],3))
    p1[:,0] = np.random.uniform(low=xmin, high=xmax, size=No[ii])
    p1[:,1] = np.random.uniform(low=0, high=ymax, size=No[ii])
    p1[:,2] = np.random.uniform(low=zmin, high=zmax, size=No[ii])

    d2o = np.linalg.norm(np.cross(p1 - Rx, Rx - Tx), axis=1) / dk0
    block_ray = blocked_rays_main_paths(p1[:,0], 
                                        p1[:,1], 
                                        Tx[0], 
                                        Rx[0], 
                                        Tx[1], 
                                        Rx[1], 
                                        r, 
                                        d2o) 
    num_obs_per_path[0,ii] = np.sum(block_ray)
    obs_pos = np.multiply(r**2 - np.square(d2o), block_ray)
    obs_len_per_path[0,ii] = np.sum(2 * np.sqrt(obs_pos),axis=0)
    
    d2o_TxRIS = np.transpose(np.multiply(np.linalg.norm(np.cross((p1 - Tx)[:, None, :], v1[None, :, :]), axis=2),inv_normv1)) 
    d2o_RISRx = np.transpose(np.multiply(np.sqrt(np.sum(np.square(np.cross((p1 - Rx)[:, None, :], v2[None, :, :])), axis=2)),inv_normv2))
    
    x1r = np.tile(np.transpose(p1[:,0]), [N*N, 1])
    y1r = np.tile(np.transpose(p1[:,1]), [N*N, 1])
    z1r = np.tile(np.transpose(p1[:,2]), [N*N, 1])    

    br_TxRIS = (x1r >=xTmin)&(x1r<=xTmax)&(y1r >=yTmin)&(y1r<=yTmax)&(z1r >=zTmin)&(z1r<=zTmax)&(d2o_TxRIS < r)    
    br_RISRx = (x1r >=xRmin)&(x1r<=xRmax)&(y1r >=yRmin)&(y1r<=yRmax)&(z1r >=zRmin)&(z1r<=zRmax)&(d2o_RISRx < r)
    
    # Obstruction length
    obs_pos_TxRIS = 2 *  np.sqrt(np.multiply(r**2 - np.square(d2o_TxRIS), br_TxRIS))
    obs_pos_RISRx = 2 *  np.sqrt(np.multiply(r**2 - np.square(d2o_RISRx), br_RISRx))
    obs_len_per_path_RIS[:,ii] = np.sum(obs_pos_TxRIS + obs_pos_RISRx, axis=1)
        
print("Elapsed time: %s seconds ---" % (time.time() - start_time))

## Probability of outage

In [ ]:
a1_exact = np.power(10, 0.1 * (Lk + z * obs_len_per_path))
Psi1_exact = a1_exact

TxRISc = [ris_center_x, ris_center_y, ris_center_z] - Tx
RxRISc = [ris_center_x, ris_center_y, ris_center_z] - Rx

cosnmTx  = np.sum([0,1,0] * (Tx - c_RIS), axis=1)/ np.linalg.norm(Tx - c_RIS, axis=1)
cosnmRx  = np.sum([0,1,0] * (Rx - c_RIS), axis=1)/ np.linalg.norm(Rx - c_RIS, axis=1)
cosTxnm  = np.sum(TxRISc * (c_RIS - Tx), axis=1) / (np.linalg.norm(TxRISc) * np.linalg.norm(c_RIS - Tx, axis=1))
cosRxnm  = np.sum(RxRISc * (c_RIS - Rx), axis=1) / (np.linalg.norm(RxRISc) * np.linalg.norm(c_RIS - Rx, axis=1))
Fcombine = np.power(cosTxnm, (Gt/2)-1) * cosnmTx * cosnmRx * np.power(cosRxnm,(Gr/2) - 1)

TxRIS_d = np.linalg.norm(v1, axis=1)
RISRx_d = np.linalg.norm(v2, axis=1)

Dair = np.sqrt(np.power(10, AO * (TxRIS_d + TxRIS_d) / 10))
num  = np.repeat(np.sqrt(Fcombine.reshape(-1,1)), trials, axis=1)

LkRIS  = 20 * np.log10(4 * np.pi/(dx**2))

D_obs_exact    = np.sqrt(np.power(10, z * obs_len_per_path_RIS / 10))
den_exact      = Dair.reshape(-1,1) * D_obs_exact * TxRIS_d.reshape(-1,1) * RISRx_d.reshape(-1,1)
RIS_term_exact = 20 * np.log10(np.absolute(np.sum(np.divide(num, den_exact), axis=0)))

a1_exact_RIS      = np.zeros((2,trials))
a1_exact_RIS[0,:] = np.power(10, 0.1 * (Lk.reshape(-1,1) + z * obs_len_per_path))
a1_exact_RIS[1,:] = np.power(10, 0.1 * (LkRIS.reshape(-1,1) - RIS_term_exact))

# Selecting the least attenuated path
Psi1_exact_RIS = a1_exact_RIS[np.argmin(a1_exact_RIS, axis=0), np.arange(0,trials)]

sqr_h = np.random.exponential(scale=1.0, size=trials)
gamma_th = np.linspace(-100, 100, 201)

compare1 = (np.divide(np.repeat(sqr_h.reshape(1,-1), len(gamma_th), axis=0),
                      np.repeat(Psi1_exact.reshape(1,-1), len(gamma_th), axis=0)).T <= B * np.power(10, 0.1 * gamma_th).reshape(1,-1))
outage1  = np.sum(compare1, axis=0) * (1 / trials)
CI1      = proportion_confint(np.sum(compare1, axis=0), trials, alpha=0.05, method='normal')

compare2 = (np.divide(np.repeat(sqr_h.reshape(1,-1), len(gamma_th), axis=0),
                      np.repeat(Psi1_exact_RIS.reshape(1,-1), len(gamma_th), axis=0)).T <= B * np.power(10, 0.1 * gamma_th).reshape(1,-1))
outage2  = np.sum(compare2, axis=0) * (1 / trials)
CI2      = proportion_confint(np.sum(compare2, axis=0), trials, alpha=0.05, method='normal')

## Figure 

In [ ]:
fig = plt.figure(dpi=150)
plt.plot(gamma_th, outage1, lw=1, label= 'LOS')
plt.fill_between(gamma_th, CI1[0], CI1[1], alpha=0.25)
plt.plot(gamma_th, outage2, '--', lw=1, label= 'RIS (%d x %d)'%(N,N))
plt.fill_between(gamma_th, CI2[0], CI2[1], alpha=0.25)

plt.xlim(-60, 100)
plt.ylim([0.001, 1])
plt.grid(which='both')
plt.yscale('log')
plt.ylabel('P_outage')
plt.xlabel('gamma_th [dBm]')
plt.title('f = %dGHz, No trials = %d, r = %.2fm'%(GHz, trials, r))
plt.legend()
plt.show()